In [2]:
from DDS_Seq import DDS, Seq
from DDS_config import *

import sys
sys.path.append("/home/xiaoshe/code/Bell/compiler/Opfunc")

from OpfuncAWG import OpfuncAWG
from OpfuncPulse import OpfuncPulse
from OpfuncRF_DDS import OpfuncRF_DDS

import re

def extract_number(string):
    # 使用正则表达式匹配所有数字
    numbers = re.findall(r'\d+', string)
    # 如果找到数字，返回第一个数字
    if numbers:
        return int(numbers[0])
    # 如果没有找到数字，返回None
    return None
# first, initialize the sequence
DDS.VERBOSE = True
# print(DDS.All)
# print(len(DDS.List))
# detection
# seq = Seq().S0(4)
seq = Seq().S0(4).S1(5)  # 序列
print(seq)
# rd_seq = seq.compile()
# print(seq.compile())
# print(seq.compile(4))
print("序列准备好了")
seqs = seq.seq
print(seqs[0][0])
print(seqs[0][0].name[0])
print(seqs[0][0][0])
print(seqs[0][0][1])
print(seqs[0][0][2])
print(extract_number("dds21"))
type(seqs[0][0])==type(DDS("dds"))

[[dds0.s0 = [100, 0.5, 0], dds1.s0 = [100, 0.5, 0], dds2.s0 = [100, 0.5, 0], dds3.s0 = [100, 0.5, 0], 2, 4], [dds12.s0 = [100, 0.2, 0], dds13.s0 = [100, 0.2, 0], dds14.s0 = [100, 0.2, 0], dds15.s0 = [100, 0.2, 0], 3, 5]]
序列准备好了
dds0.s0 = [100, 0.5, 0]
dds0
100
0.5
0
21


True

In [3]:
dds = [OpfuncRF_DDS(DeviceID=i) for i in range(16)]
ttl = [OpfuncPulse(DeviceID=i) for i in range(16)]

In [5]:
def compile2Bell(seqs): # RTMQv1.0 transfer to Bell
    """
    将序列转换为Bell硬件执行的IR   
    每个dds通道有4个参数[f,a,p,t]
    每个ttl通道有3个参数[mode,address]，暂时用来计数input , [mode, delay]
    """
    seq_len = len(seqs)
    seq4Bell = []
    for seq in seqs:
        # 逆序遍历
        Flag_TTL = False
        Flag_Delay = False
        delay = 0
        DDSList = []
        for item in seq[::-1]:
            if type(item)==type(DDS("dds")):
                # print("dds")
                DeviceID = extract_number(item.name[0])
                Freq = item[0]
                Amp = item[1]
                Phase = item[2]
                DDSList.append(["dds",DeviceID, Freq, Amp, Phase,delay])
            elif Flag_Delay == False:
                delay = item # 读取延时
                Flag_Delay = True
                # print("delay:", delay)
            elif Flag_TTL==False: # 读取TTL
                TTL = item
                mode = 1
                # if mode == #TTL mode 在硬件中不能导入
                Flag_TTL = True

                # print("TTL channel:", TTL)
        DDSList.reverse() # 反转顺序
        DDSList.append(["TTL",TTL, mode, delay])
        # print("DDSList:", DDSList)
        seq4Bell.append(DDSList)
    # print("Bell序列:", seq4Bell)
    return seq4Bell
seqsIR = compile2Bell(seqs)
print("Bell序列:", seqsIR)

Bell序列: [[['dds', 0, 100, 0.5, 0, 4], ['dds', 1, 100, 0.5, 0, 4], ['dds', 2, 100, 0.5, 0, 4], ['dds', 3, 100, 0.5, 0, 4], ['TTL', 2, 1, 4]], [['dds', 12, 100, 0.2, 0, 5], ['dds', 13, 100, 0.2, 0, 5], ['dds', 14, 100, 0.2, 0, 5], ['dds', 15, 100, 0.2, 0, 5], ['TTL', 3, 1, 5]]]


In [9]:
# IR to Bell hardware 
"""
这个函数用来将IR数据结构转换成Bell机器码，暂不考虑跳转格式
"""
print(seqsIR)
dds = [OpfuncRF_DDS(DeviceID=i) for i in range(16)]
ttl = [OpfuncPulse(DeviceID=i) for i in range(16)]

[[['dds', 0, 100, 0.5, 0, 4], ['dds', 1, 100, 0.5, 0, 4], ['dds', 2, 100, 0.5, 0, 4], ['dds', 3, 100, 0.5, 0, 4], ['TTL', 2, 4]], [['dds', 12, 100, 0.2, 0, 5], ['dds', 13, 100, 0.2, 0, 5], ['dds', 14, 100, 0.2, 0, 5], ['dds', 15, 100, 0.2, 0, 5], ['TTL', 3, 5]]]


In [ ]:
for seq in range(seqsIR):
    for item in seq:
        if item[0] == "dds":
            dds[item[1]].setwaveform(item[2], item[3], item[4], item[5])
        elif item[0] == "TTL":